In [1]:
import numpy as np

In [2]:
# 特征0：0：青年，1：中年，2：老年
# 特征1：0：无工作，1：有工作
# 特征2：0：无房产，1：有房产
# 特征3：0：信贷一般，1：信贷好，2：信贷非常好
X = np.array([[0,0,0,0],
              [0,0,0,1],
              [0,1,0,1],
              [0,1,1,0],
              [0,0,0,0],
              [1,0,0,0],
              [1,0,0,1],
              [1,1,1,1],
              [1,0,1,2],
              [1,0,1,2],
              [2,0,1,2],
              [2,0,1,1],
              [2,1,0,1],
              [2,1,0,2],
              [2,0,0,0]])
# 标签：0：不贷款，1：贷款
y = np.array([0,0,1,1,0,0,0,1,1,1,1,1,1,1,0])

In [3]:
# 特征和标签的可取值范围：
def H(y):
    sum = 0
    # 计算y可取到的值
    k = set(y)
    for ck in k:
        Pk = y[y==ck].shape[0] / y.shape[0]
        if Pk != 0:
            sum -= Pk * np.log2(Pk)
    return sum

In [4]:
def multi(y):
    ySet = set(y)
    bestCount = 0
    for yi in ySet:
        count = y.count(yi)
        if count > bestCount:
            bestCount = count
            bestyi = yi
    return bestyi

In [34]:
def gRDA(X, y, feature):
    # 计算X的每个特征可取到的值
    a = set(X[:,feature])
    # 计算数据集的经验熵
    HD = H(y)
    # 计算特征A对数据集D的经验条件熵H(D|A)
    HDA = 0
    for value in a:
        yDi = y[X[:,feature]==value]
        HDA += yDi.shape[0]/y.shape[0] * H(yDi)
    return (HD - HDA)/HD

def C45(X, y, epsilon):
    # 若D中所有实例属于同一类
    if len(set(y))==1:
        # 将类$$C_k$$作为该结点的类标记
        return {'entropy':0, 'label':y[0], 'Nt':X.shape[0]}
    # 若$$A=\emptyset$$
    if X.shape[1] == 0:
        # 实例数最大的类$$C_k$$作为该结点的类标记
        return {'entropy':H(y), 'label':multi(y), 'Nt':X.shape[0]}
    bestInfo = 0
    # 计算A中各个特征对D的信息增益
    for feature in range(X.shape[1]):
        info = gRDA(X, y, feature)
        # 选择信息特征最大的Ag
        if info > bestInfo:
            bestInfo = info
            bestfeature = feature
    # 如果Ag的信息增益小于阈值$$\epsilon$$
    if bestInfo < epsilon:
        # 将D中实例数最大的类$$C_k$$作为该结点的类标记
        return multi(y)
    feature = bestfeature
    ret = {'feature':feature, 'entropy':H(y), 'Nt':X.shape[0], 'child':{}}
    # 对Ag的每一个可能的值ai
    a = set(X[:, feature])
    for ai in a:
        yai = y[X[:,feature] == ai]
        Xai = X[X[:,feature] == ai]
        ret['child'][ai] = C45(Xai, yai, epsilon)
    return ret

In [35]:
C45(X, y, 0)

{'feature': 2,
 'entropy': 0.9709505944546686,
 'Nt': 15,
 'child': {0: {'feature': 1,
   'entropy': 0.9182958340544896,
   'Nt': 9,
   'child': {0: {'entropy': 0, 'label': 0, 'Nt': 6},
    1: {'entropy': 0, 'label': 1, 'Nt': 3}}},
  1: {'entropy': 0, 'label': 1, 'Nt': 6}}}

In [50]:
def isTree(Node):
    #print (Node)
    return 'child' in Node.keys()

def Clip(Node):
    bestNt = 0
    for value in Node['child']:
        if Node['child'][value]['Nt'] > bestNt:
            bestNt = Node['child'][value]['Nt']
            bestLabel = Node['child'][value]['label']
    Node['label'] = bestLabel
    Node.pop('child')
    
def Merge(Node, alpha):
    # 计算CaTb
    CT_b = 0
    for value in Node['child']:
        CT_b = CT_b + Node['child'][value]['Nt'] * Node['child'][value]['entropy'] + alpha
    # 计算CaTa
    CT_a = Node['entropy'] + alpha
    # 剪枝的条件
    if CT_a <= CT_b:
        Clip(Node)
        
def prune(Node, alpha):
    # 判断子结点中是否存在树
    for value in Node['child']:
        # 如果存在树
        if isTree(Node['child'][value]):
            # 先对树子结点做prune
            prune(Node['child'][value], alpha)
    # 对所有子树都prune之后，判断是否所有子树都是叶子
    isAllLeaf = True
    for value in Node['child']:
        if isTree(Node['child'][value]):
            isAllLeaf = False
            break
    # 如果所有子树都是叶子
    if isAllLeaf:
        # 尝试对结点做剪枝
        Merge(Node, alpha)

In [53]:
tree = C45(X, y, 0)
print (tree)
prune(tree, 10)
print (tree)

{'feature': 2, 'entropy': 0.9709505944546686, 'Nt': 15, 'child': {0: {'feature': 1, 'entropy': 0.9182958340544896, 'Nt': 9, 'child': {0: {'entropy': 0, 'label': 0, 'Nt': 6}, 1: {'entropy': 0, 'label': 1, 'Nt': 3}}}, 1: {'entropy': 0, 'label': 1, 'Nt': 6}}}
{'feature': 2, 'entropy': 0.9709505944546686, 'Nt': 15, 'label': 0}
